[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/KedoKudo/DT_GNN_Tutorial/blob/tree/main/notebooks/07_pytorch_geometric_temporal_intro.ipynb)

# Introduction to PyTorch Geometric Temporal

Time series prediction using graphs leverages spatial information (topology) with temporal (time-based) data, leading to better forecasting in many domains, from traffic flow in urban cities to protein interactions over time.
This notebook delves into graph time series prediction using the PyTorch Geometric Temporal library.

## Table of Contents

1. [Loading the Dataset](#Loading-the-Dataset)
2. [Defining the Model](#Defining-the-Model)
3. [Setting up the Training Loop](#Setting-up-the-Training-Loop)
4. [Evaluating Forecasting Results](#Evaluating-Forecasting-Results)

In [1]:
# Uncomment the following line to install PyTorch Geometric Temporal
# !pip install torch torch-geometric torch-geometric-temporal

## Loading the Dataset <a name="Loading-the-Dataset"></a>

PyTorch Geometric Temporal provides spatiotemporal datasets tailored for graph time series forecasting.

In [2]:
from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split

loader = ChickenpoxDatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

## Defining the Model <a name="Defining-the-Model"></a>

We'll use a recurrent graph convolution layer followed by an LSTM for time series forecasting.

In [3]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN

class SpatioTemporalPredictor(torch.nn.Module):
    def __init__(self, node_features):
        super(SpatioTemporalPredictor, self).__init__()
        self.recurrent = DCRNN(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        return self.linear(h)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SpatioTemporalPredictor(node_features = 4).to(device)

> m1 chip's mps is not supported by pytorch geometric temporal yet due to missing support for int64 from PyTorch, therefore it is necessary to use cpu for this notebook when running on mac.

## Setting up the Training Loop <a name="Setting-up-the-Training-Loop"></a>

Let's train our spatiotemporal forecasting model.

Due to nature of the time sequence, the training loop is effectively a sliding window over the time series.
And here we are accumulating the loss over the entire time series before updating the model parameters.
There are other researchers prefer updating the model parameters at each time step, which is also possible with PyTorch Geometric Temporal.
It is your job to decide which approach is better for your particular problem, as there is no one-size-fits-all solution here.

In [4]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in range(200):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        snapshot = snapshot.to(device)
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

    # Print loss for every 10 epochs
    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, Loss: {cost.item()}')

Epoch: 0, Loss: 0.9830449819564819
Epoch: 10, Loss: 0.9693251848220825
Epoch: 20, Loss: 0.9669546484947205
Epoch: 30, Loss: 0.9650921821594238
Epoch: 40, Loss: 0.9636843204498291
Epoch: 50, Loss: 0.9628620743751526
Epoch: 60, Loss: 0.9621609449386597
Epoch: 70, Loss: 0.9615795612335205
Epoch: 80, Loss: 0.9611184000968933
Epoch: 90, Loss: 0.9607266187667847
Epoch: 100, Loss: 0.9603118300437927
Epoch: 110, Loss: 0.9599068760871887
Epoch: 120, Loss: 0.9594556093215942
Epoch: 130, Loss: 0.9590811729431152
Epoch: 140, Loss: 0.9587007164955139
Epoch: 150, Loss: 0.9583893418312073
Epoch: 160, Loss: 0.9581165909767151
Epoch: 170, Loss: 0.9577996730804443
Epoch: 180, Loss: 0.9575451016426086
Epoch: 190, Loss: 0.957292914390564


## Evaluating Forecasting Results <a name="Evaluating-Forecasting-Results"></a>

It's time to evaluate the model's forecasting capability on the test set.

In [5]:
model.eval()
cost = 0
for time, snapshot in enumerate(test_dataset):
    snapshot = snapshot.to(device)
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

MSE: 1.0552


For better prediction, one needs to increase the model capacity, train for longer, and use a more sophisticated model.
Furthermore, one can also use Optuna to tune the hyperparameters of the model can lead to better results.